In [5]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
import time

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

# from sympy import *
from scipy.linalg import schur, eigvals
from scipy.special import comb, perm

extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [7]:
def generate_localstatsmat_eig_desym(Nparams,Grand,rho):
    '''
    Input
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    xaxiss,yaxiss =np.arange(NE*2),np.arange(NE*2)
    Xax,Yax=np.meshgrid(xaxiss,yaxiss)
    idxwhere = np.where(Xax>Yax)
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    X,Xsym,Xasym=np.zeros((N,N)),np.zeros((N,N)),np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    XT = X.copy().T

    Xsym = np.sqrt(rho)*X.copy()
    Xsym[idxwhere]=np.sqrt(rho)*XT[idxwhere]
    
    Xasym=np.sqrt(1-rho)*np.random.randn(NE*2,NE*2)
    X=Xsym.copy()+Xasym.copy()
    # components
    Xsym[:NE,:NE],Xsym[NE:,NE:]=xee*Xsym[:NE,:NE]/np.sqrt(NE),xii*Xsym[NE:,NE:]/np.sqrt(NI)
    Xsym[:NE,NE:],Xsym[NE:,:NE]=xei*Xsym[:NE,NE:]/np.sqrt(NE),xie*Xsym[NE:,:NE]/np.sqrt(NI)
    Xasym[:NE,:NE],Xasym[NE:,NE:]=xee*Xasym[:NE,:NE]/np.sqrt(NE),xii*Xasym[NE:,NE:]/np.sqrt(NI)
    Xasym[:NE,NE:],Xasym[NE:,:NE]=xei*Xasym[:NE,NE:]/np.sqrt(NE),xie*Xasym[NE:,:NE]/np.sqrt(NI)
    X=Xsym.copy()+Xasym.copy()
    # theoretical M (grandom) matrix
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=la.eig(gmat)
    
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii
    return (X,Xsym,Xasym,Gamp,eigvgm,eigvecgm,gaverage)

def generate_localstatsmat_eig(Nparams,Grand):
    '''
    Input 
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nparams/N   
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    eigvgm,eigvecgm=la.eig(gmat)
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage)

def generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nparams/N   
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI
    # ua,svam,vha=la.svd(Am)
    # va=vha.T
    # svam=svam[0]
    # ua,va=np.zeros((N,N)),np.zeros((N,N))
    # ua[:,0]=1.0/np.sqrt(N)
    # va[:NE,0]=JE/np.sqrt(NE)
    # va[NE:,0]=-JI/np.sqrt(NI)
    # va[:,0]/=-np.linalg.norm(va[:,0])
    # svam=np.sqrt(2*(JE**2+JI**2))
    # vha=va.T

    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JEE,JEI,JIE,JII

    return (Am,Jsv)#,ua,svam,va)

# generate gmat, same gavg
def gmatamplitude_eig(gavgfix,typenum):
    Amplit = gavgfix*typenum
    numsample = typenum**2
    Amplitg= np.zeros(numsample)
    idxc=0
    while (1):
      if idxc>=numsample:
        Amplitg[numsample-1]=1.0-np.sum(Amplitg[:numsample-1])
        break
      p=np.random.random(1)
      Amplitg[idxc]=np.minimum(p,1.0-np.sum(Amplitg))
      if np.sum(Amplitg)>1.0:
        continue 
      elif np.sum(Amplitg)==1.0:
        break
      else:
        idxc +=1
        # Amplitg[idxc]=np.min(p,1.0-np.sum(Amplitg))
    # Amplitg=0
    Amplitg*=Amplit
    Amplitg=np.sqrt(Amplitg)
    return Amplitg

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    print(np.shape(cov))
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

In [8]:
def gradtanh(xorg):
    gradx = xorg.copy()
    nneuron,nknum=np.shape(gradx)[0],np.shape(gradx)[1]
    for i in range(nneuron):
        for j in range(nknum):
            gradx[i,j]=4/(np.exp(-xorg[i,j])+np.exp(xorg[i,j]))**2
    return gradx

## Random Perturbation
* single outlier

In [6]:
'''
WITH SYMMETRY
'''
'''
Cases 3,4 with symmetry
Focus on mean 
Focus on second moment
Focus on derivative on 
'''
# generate mean matrix
nrank=1
Nt=np.array([750,750])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
nrank,ntrial,neta,ngavg=1,10,10,10

## for all casesq
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)

''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

''' ---------- method 2 ------------------ '''
''' ## Three \bar{J} cases '''
JI,JE,a,b=0.575,1.225,0,0
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
''' for reference '''
xAm,yAm=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))


''' Network Setting for Iterating Following '''
## heterogeneous random gain
''' >>>>>>>>>>>>>> Case 1 and Case 2 >>>>>>>>>>>>>>>>>>>>>>> '''
gaverage = 1.0
xee,xei,xie,xii=1.0,1.0,1.0,1.0 
coeffetaEsub=np.array([1.0,1.0,0.0])#np.array([0.6,0.0,0.2])
coeffetaTotal = np.array([1.0,1.0,1.0])#
# coeffetaTotal = np.zeros(3)
signetaEsub=np.ones(3)
signetaTotal=np.ones(3)
# signetaTotal[:]=-1
# signetaTotal[1]=+1

ppercentEsub = np.ones(2)
ppercentEsub[0]=0.5
ppercentEsub[1]=1.0-ppercentEsub[0]

'''  Recording Variables. '''
# original data
Reigvecseries0,Leigvecseries0=np.zeros((ntrial,NE*2),dtype=complex),np.zeros((ngavg,ntrial,NE*2),dtype=complex)
Beigvseries0=np.zeros((ntrial,N),dtype=complex)
x0series,y0series=np.zeros((ntrial,NE*2),dtype=complex),np.zeros((ntrial,NE*2),dtype=complex)
nknum = 40
muphiseries,muderivphiseries = np.zeros((ntrial,2,nknum)),np.zeros((ntrial,2,nknum))
kseries = np.reshape(np.linspace(-4.,4.,nknum),(1,nknum))
derivativephi = np.zeros((ntrial,2))
transferfun = 'tanh'

rhsfun = np.zeros((ntrial,nknum))

''' All random matrices for each trials '''
## Not X total (symmetry), nut Y independent
gYbasexee=np.zeros((ntrial,NE*2,NE*2),dtype=complex)
''' Iterative Processing '''
eta=1.0
coeffetaIseries = np.linspace(0.0,1.0,neta)
for iktrial in range(ntrial):
    '''    ##>>>>>>>>>>> g0!=0, >>>>>>>>     '''
    Xsym=np.random.randn(NE*2,NE*2)*gaverage/np.sqrt(2*NE) # /np.sqrt(2) no later
    Xsym[:NE,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[:NE,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[NE:,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[NE:,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    XsymT=Xsym.copy().T
    X0=Xsym.copy()
    J0=Am.copy()+X0.copy()
    ''' # calculate the original \bar{m} and \bar{n} '''
    eigvJ0,eigrvecJ0=la.eig(J0)
    inveigrvecJ0=la.inv(eigrvecJ0)
    meig0 = np.squeeze(eigrvecJ0[:,:].copy())
    neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
    neig0=neig0.copy().T
    print('test normal:',np.sum(neig0[:,0]*meig0[:,0]))
    for i in range(nrank):
        neig0[:,i]*=eigvJ0[i]
    for j in range(1,N):
        neig0[:,j]*=eigvJ0[j]

    leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
    ''' The first '''
    reig0=meig0.copy()
    normval=np.sum(reig0*neig0.copy(),axis=0)
    normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
    leig0=neig0.copy()/normval.copy()

    print('eigenvalues:',eigvJ0[:nrank])
    '''## for reference '''
    tildex,tildey=np.reshape(reig0[:,0].copy(),(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
    # ''' ## normalization 0 (reference) absolute reference is xAm, yAm '''
    # tildex,tildey=np.reshape(x0[:,0].copy(),(NE*2,1)),np.reshape(y0[:,0].copy(),(NE*2,1))
    ## make sure the E in x is positive, if negative then all x and y should change sign
    if (np.mean(tildey[:NE,0])*yAm[0,0])<0:
        tildex*=(-1)
        tildey*=(-1)
    x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
    x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
    y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey

    x0,y0=np.squeeze(x0),np.squeeze(y0)
    x0series[iktrial,:],y0series[iktrial,:]=x0.copy(),y0.copy()
    ''' record the random derivative '''
    print('recording!!!')
    ## log: there is a bug, X is the total random xconn, including symmetry
    gYbasexee[iktrial,:,:]=Xsym.copy()

    ''' kappa dynamics    '''
    phie, phii = np.reshape(x0[:NE],(NE,1)),np.reshape(x0[NE:],(NE,1))
    if transferfun =='tanh':
        muphiseries[iktrial,0,:]= np.mean(np.tanh(phie@kseries),axis=0)[:]
        muphiseries[iktrial,1,:]= np.mean(np.tanh(phii@kseries),axis=0)[:]
    rhs = 2*JE/np.sqrt(NE*2)/(JE-JI)*muphiseries[iktrial,0,:]*NE-2*JI/np.sqrt(NE*2)/(JE-JI)*muphiseries[iktrial,1,:]*NE
    rhsfun[iktrial,:] = rhs *(eigvAm[0])
        

    
    


test normal: (0.9999999999999997-4.4162590251417555e-14j)
<ipython-input-6-d0ff9051b087>:126: ComplexWarning: Casting complex values to real discards the imaginary part
  Z[i,i]=1/(b1-eigvJ0[0])
test normal 2: (0.9999999999999989+5.100087019371813e-16j)
test normal 3: (-0.511700396698098+0.8646830580615723j)
eigenvalues: [-0.5117004+0.86468306j]
recording!!!
<ipython-input-6-d0ff9051b087>:144: ComplexWarning: Casting complex values to real discards the imaginary part
  x0series[iktrial,:],y0series[iktrial,:]=x0.copy(),y0.copy()
<ipython-input-6-d0ff9051b087>:153: ComplexWarning: Casting complex values to real discards the imaginary part
  muphiseries[iktrial,0,:]= np.mean(np.tanh(phie@kseries),axis=0)[:]
<ipython-input-6-d0ff9051b087>:154: ComplexWarning: Casting complex values to real discards the imaginary part
  muphiseries[iktrial,1,:]= np.mean(np.tanh(phii@kseries),axis=0)[:]
<ipython-input-6-d0ff9051b087>:156: ComplexWarning: Casting complex values to real discards the imaginary 

In [7]:
iktrial=8
fig,ax=plt.subplots(figsize=(4,4))  
ax.plot(kseries[0,:],kseries[0,:],linestyle='--',color='grey')
ax.plot(kseries[0,:],rhsfun[iktrial,:],color='k',linewidth=1.5)


In [11]:
## test analytical solution for < Gaussian >
sigmaxg = 3.0/np.sqrt(2*NE)
xg = np.linspace(1/np.sqrt(NE*2)-1,1/np.sqrt(2*NE)+1)
rhoxg = np.exp(-(xg-1/np.sqrt(NE*2))**2/sigmaxg)
rhoxg = rhoxg/np.sum(rhoxg)
mutestphi = np.sum(np.tanh(xg)*rhoxg)
print('test meanPhi:',mutestphi,', avg:',np.tanh(1/np.sqrt(2*NE)))

test meanPhi: 0.02488490816206802 , avg: 0.025814152751265352


In [69]:
'''
WITH SYMMETRY
'''
'''
* multi-population (example 2 E and 1 I)
* homogeneous random gain g
* heterogeneous degree of symmetry
'''
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
nrank,ntrial,neta,nvec,nmaxFP=1,3,11,2,3

## for all casesq
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)

''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

''' ---------- method 2 ------------------ '''
''' ## Three \bar{J} cases '''
JI,JE,a,b=3.575,4.425,0,0
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
''' for reference, eigenvectors of unperturbed matrix \bar{J}'''
xAm,yAm=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))


''' Network Setting  '''
## heterogeneous random gain
gaverage = 0.8
xee,xei,xie,xii=1.0,1.0,1.0,1.0 

## heterogeneous degree of symmetry: amplitudes and signs
coeffetaEsub=np.array([1.0,1.0,1.0])#
coeffetaTotal = np.array([1.0,1.0,1.0])#
# coeffetaTotal = np.zeros(3)
signetaEsub=np.ones(3)
signetaTotal=np.ones(3)
signetaTotal[2]=-1
# signetaTotal[:]=-1
# signetaTotal[1]=+1

ppercentEsub = np.ones(2)
ppercentEsub[0]=0.5
ppercentEsub[1]=1.0-ppercentEsub[0]
## E->total ppercentEsub[0]/2.0,ppercentEsub[1]/2.0, (I) 1/2.0

'''  Recording Variables. '''
## perturbed + symmetric eigenvectors
Reigvecseries,Leigvecseries=np.zeros((neta,ntrial,NE*2,nvec*2),dtype=complex),np.zeros((neta,ntrial,NE*2,nvec*2),dtype=complex)
Beigvseries=np.zeros((neta,ntrial,N),dtype=complex)
## corresponding statistics \mu and \sigma \cov
armu,sigrcov = np.zeros((neta,ntrial,2),dtype=complex),np.zeros((neta,ntrial,2),dtype=complex) # 2 for E and I
almu,siglcov = np.zeros((neta,ntrial,2),dtype=complex),np.zeros((neta,ntrial,2),dtype=complex)
sigrl = np.zeros((neta,ntrial,2),dtype=complex)
## reference, eigenvectors of matrix with iid perturbations
x0series,y0series=np.zeros((ntrial,NE*2),dtype=complex),np.zeros((ntrial,NE*2),dtype=complex)

''' Nullclines parameters  '''
nknum = 81
kseries = np.reshape(np.linspace(-4.,4.,nknum),(1,nknum))
transferfun = 'tanh'

nullclines_rhsfun = np.zeros((neta,ntrial,nknum),dtype=complex)
kappaintersect    = np.zeros((neta,ntrial,nmaxFP*2))

''' simulating parameters '''
tt = np.linspace(0,100,500)
dt = tt[2]-tt[1]
ntt = len(tt)
xfpseries = np.zeros((neta,ntrial,NE*2,ntt))
''' All random matrices for each trials '''
## Not X total (symmetry), nut Y independent
gYbasexee=np.zeros((ntrial,NE*2,NE*2),dtype=complex)
''' Iterative Processing '''
etaseries = np.linspace(0.0,1.0,neta)
# if eta == 0.0:
for iktrial in range(ntrial):
    '''    ##>>>>>>>>>>> g0!=0, >>>>>>>>     '''
    Xsym=np.random.randn(NE*2,NE*2)*gaverage/np.sqrt(2*NE) # /np.sqrt(2) no later
    Xsym[:NE,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[:NE,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[NE:,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[NE:,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    XsymT=Xsym.copy().T
    X0=Xsym.copy()
    J0=Am.copy()+X0.copy()
    ''' # calculate the original \bar{m} and \bar{n} '''
    eigvJ0,eigrvecJ0=la.eig(J0)
    inveigrvecJ0=la.inv(eigrvecJ0)
    meig0 = np.squeeze(eigrvecJ0[:,:].copy())
    neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
    neig0=neig0.copy().T
    for i in range(nrank):
        neig0[:,i]*=eigvJ0[i]
    for j in range(nrank,N):
        neig0[:,j]*=eigvJ0[j]

    leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
    ''' The first '''
    reig0=meig0.copy()
    normval=np.sum(reig0*neig0.copy(),axis=0)
    normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
    leig0=neig0.copy()/normval.copy()
    ## for reference
    tildex,tildey=np.reshape(reig0[:,0].copy(),(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
    ## make sure the E in y is positive, for negative change signs of x and y
    if (np.mean(tildey[:NE,0])*yAm[0,0])<0:
        tildex*=(-1)
        tildey*=(-1)
    x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
    x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
    y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey

    x0series[iktrial,:],y0series[iktrial,:]=x0[:,0].copy(),y0[:,0].copy()
    ''' record the random derivative '''
    print('recording!!!')
    ## log: there is a bug, X is the total random xconn, including symmetry
    gYbasexee[iktrial,:,:]=Xsym.copy()
    for idxeta, eta in enumerate(etaseries):
        etaset=eta*np.ones(6)
        for iloop in range(3):
            etaset[iloop] = etaset[iloop]*coeffetaEsub[iloop]
            etaset[iloop+3] = etaset[iloop+3]*coeffetaTotal[iloop]
        print('Total Symmetry:',etaset)
        Xinit = Xsym.copy()
        ### >>>>>>>>.Subcircuit Exc sym >>>>>>>>>.
        ## remaining
        hNE=int(ppercentEsub[0]*NE) 
        asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[:hNE,:hNE]=signetaEsub[0]*aamp*XsymT[:hNE,:hNE].copy()+np.sqrt(1-aamp**2)*Xsym[:hNE,:hNE].copy()
        ## >>>>>>. esub middle
        asqr=(1-np.sqrt(1-etaset[1]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[:hNE,hNE:NE]=signetaEsub[1]*aamp*XsymT[:hNE,hNE:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:hNE,hNE:NE].copy()
        Xinit[hNE:NE,:hNE]=signetaEsub[1]*aamp*XsymT[hNE:NE,:hNE].copy()+np.sqrt(1-aamp**2)*Xsym[hNE:NE,:hNE].copy()
        ## >>>>>> E2
        asqr=(1-np.sqrt(1-etaset[2]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[hNE:NE,hNE:NE]=signetaEsub[2]*aamp*XsymT[hNE:NE,hNE:NE].copy()+np.sqrt(1-aamp**2)*Xsym[hNE:NE,hNE:NE].copy()
        ### >>>>>>> Total >>>>>
        # E1I-B
        hNE = int(ppercentEsub[0]*NE) 
        asqr=(1-np.sqrt(1-(etaset[3])**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[NE:,:hNE]=signetaTotal[0]*aamp*XsymT[NE:,:hNE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:hNE].copy()
        Xinit[:hNE,NE:]=signetaTotal[0]*aamp*XsymT[:hNE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[:hNE,NE:].copy()
        # E2I-B
        asqr=(1-np.sqrt(1-(etaset[4])**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[NE:,hNE:NE]=signetaTotal[1]*aamp*XsymT[NE:,hNE:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,hNE:NE].copy()
        Xinit[hNE:NE,NE:]=signetaTotal[1]*aamp*XsymT[hNE:NE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[hNE:NE,NE:].copy()
        ## II ##
        asqr=(1-np.sqrt(1-etaset[5]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,NE:]=signetaTotal[2]*aamp*XsymT[NE:,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,NE:].copy()
                    
        X=Xinit.copy()
        X[:NE,:NE]*=(xee)
        X[:NE,NE:]*=(xei)
        X[NE:,:NE]*=(xie)
        X[NE:,NE:]*=(xii)

        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
        symnumfull*=2*NE
        print('compare degree of symmetry:',symnumfull,'--num; ')#,etaset*signeta,'--setting')
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        # meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
        for j in range(nrank,N):
            neig[:,j]*=eigvJ[j]
        '''    Sort Eigenvalue in ascending     '''
        eigenvalue_Rsort = np.squeeze(-np.abs(np.real(eigvJ.copy())))
        idxsort=np.argsort(eigenvalue_Rsort)
        eigenvalue_sort = np.squeeze(eigvJ.copy())
        eigvJ= eigvJ[idxsort]  #### >>>>>>>>>>>>>> resorting >>>>>>>>>
        ## >>> for eigendecomposition 
        Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
        Reig = meig[:,idxsort]
        Leig = neig[:,idxsort]
        normval=np.sum(Reig*Leig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        Leig=Leig.copy()/normval.copy()   # error
        '''   normalization 0 (reference)   '''
        for irec in range(1):
            tildex,tildey=np.reshape(Reig[:,irec].copy(),(2*NE,1)),np.reshape(Leig[:,irec].copy(),(NE*2,1))
            if (np.mean(tildey[:NE,0])*yAm[0,0])<0:
                tildex*=(-1)
                tildey*=(-1)
            xnorm0=np.sqrt(np.squeeze(tildey.T@x0)/np.squeeze(y0.T@tildex))*tildex
            ynorm0=np.sqrt(np.squeeze(tildex.T@y0)/np.squeeze(x0.T@tildey))*tildey
            # xrand0,yrand0=xnorm0.copy(),ynorm0.copy()
            # xrand0[:NE,0],xrand0[NE:,0]=xnorm0[:NE,0]-np.mean(xnorm0[:NE,0]),xnorm0[NE:,0]-np.mean(xnorm0[NE:,0])
            # yrand0[:NE,0],yrand0[NE:,0]=ynorm0[:NE,0]-np.mean(ynorm0[:NE,0]),ynorm0[NE:,0]-np.mean(ynorm0[NE:,0])
            # normrande,normrandi = np.std(xrand0[:NE,0]),np.std(xrand0[NE:,0])
            # xrand0[:NE,0],yrand0[:NE,0] = xrand0[:NE,0]/normrande,yrand0[:NE,0]*normrande
            # xrand0[NE:,0],yrand0[NE:,0] = xrand0[NE:,0]/normrandi,yrand0[NE:,0]*normrandi
            # xnorm0[:NE,0],xnorm0[NE:,0]=np.mean(xnorm0[:NE,0])+xrand0[:NE,0],np.mean(xnorm0[NE:,0])+xrand0[NE:,0]
            # ynorm0[:NE,0],ynorm0[NE:,0]=np.mean(ynorm0[:NE,0])+yrand0[:NE,0],np.mean(ynorm0[NE:,0])+yrand0[NE:,0]

            '''
            ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N 
            '''
            xnorm0,ynorm0 = xnorm0*np.sqrt(2*NE),ynorm0*np.sqrt(2*NE)
            Reigvecseries[idxeta,iktrial,:,irec],Leigvecseries[idxeta,iktrial,:,irec]=xnorm0[:,0].copy(),ynorm0[:,0].copy()
            ## statistics mean(mu) and variance(std/var)
            armu[idxeta,iktrial,0],armu[idxeta,iktrial,1] = np.mean(xnorm0[:NE,0]),np.mean(xnorm0[NE:,0])
            sigrcov[idxeta,iktrial,0],sigrcov[idxeta,iktrial,1] = np.std(xnorm0[:NE,0])**2,np.std(xnorm0[NE:,0])**2
            testsigrcove,testsigrcovi = np.sum((Reigvecseries[idxeta,iktrial,:NE,irec]-armu[idxeta,iktrial,0])**2)/NE,np.sum((Reigvecseries[idxeta,iktrial,NE:,irec]-armu[idxeta,iktrial,1])**2)/NE
            # print('compare single sigma:',sigrcov[idxeta,iktrial,:],'; testmanu:',testsigrcove,testsigrcovi)
            almu[idxeta,iktrial,0],almu[idxeta,iktrial,1] =np.mean(ynorm0[:NE,0]),np.mean(ynorm0[NE:,0])
            siglcov[idxeta,iktrial,0],siglcov[idxeta,iktrial,1] = np.std(ynorm0[:NE,0])**2,np.std(ynorm0[NE:,0])**2
            rlcove = (xnorm0[:NE,0]-np.mean(xnorm0[:NE,0]))*(ynorm0[:NE,0]-np.mean(ynorm0[:NE,0]))
            sigrl[idxeta,iktrial,0] = np.sum(rlcove)/NE
            rlcovi = (xnorm0[NE:,0]-np.mean(xnorm0[NE:,0]))*(ynorm0[NE:,0]-np.mean(ynorm0[NE:,0]))
            sigrl[idxeta,iktrial,1] = np.sum(rlcovi)/NE

            # print('compare e:',np.mean(rlcove),';', sigrl[idxeta,iktrial,0])
            # print('compare i:',np.mean(rlcovi),';', sigrl[idxeta,iktrial,1])
        
        ''' test reconstruction '''
        rtest,ltest = np.squeeze(Reigvecseries[idxeta,iktrial,:,0].copy()),np.squeeze(Leigvecseries[idxeta,iktrial,:,0].copy())
        rtest,ltest = np.reshape(np.squeeze(rtest),(2*NE,1)),np.reshape(np.squeeze(ltest),(2*NE,1))
        Jtest = rtest@ltest.T/NE/2*eigvJ[0]
        eigvtest,_ = la.eig(Jtest)
        
        ''' overlap mat's eigenvalue should be the same '''
        eigvov = 0.5*(armu[idxeta,iktrial,0]*almu[idxeta,iktrial,0]+sigrl[idxeta,iktrial,0])
        eigvov += 0.5*(armu[idxeta,iktrial,1]*almu[idxeta,iktrial,1]+sigrl[idxeta,iktrial,1])
        eigvov *=eigvJ[0]
        print('compare the accuracy:',eigvJ[0],'; test:',eigvtest[0],'; overlap:',eigvov)
        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()

        ## Rank One Approximation, Nullclines of \kappa
        xnorm0, ynorm0 = Reigvecseries[idxeta,iktrial,:,0].copy(),Leigvecseries[idxeta,iktrial,:,0]
        xnorm0, ynorm0 = np.reshape(xnorm0,(2*NE,1)),np.reshape(ynorm0,(2*NE,1))
        xmu,ymu = armu[idxeta,iktrial,:].copy(),almu[idxeta,iktrial,:].copy()
        xsig,ysig = sigrcov[idxeta,iktrial,:].copy(),siglcov[idxeta,iktrial,:].copy()
        xycov = sigrl[idxeta,iktrial,:].copy()
        ''' kappa dynamics    '''

        ## overlap theoretical values
        overlaptheo = np.zeros(2,dtype=complex)
        overlaptheo[0]=2*JE*etaset[0]*signetaEsub[0]/2.0/(JE-JI)-2*JI*etaset[3]*signetaTotal[0]/2.0/(JE-JI)#gaverage**2/eigvAm[0]**2*eta
        overlaptheo[1]=2*JE*etaset[4]*signetaTotal[1]/2.0/(JE-JI)-2*JI*etaset[5]*signetaTotal[2]/2.0/(JE-JI)
        # xycov[0],xycov[1]=overlaptheo[0],overlaptheo[1]
        print('overlap, theo:',overlaptheo,'; num:',xycov)

        ## 2 populations 
        ## phie and phii, mu phie mu phii, derivative phie phii, mu derivative phie phii
        muphifunc,muderivativephifunc = np.zeros((2,nknum)),np.zeros((2,nknum))
        phie, phii = np.reshape(xnorm0[:NE,0],(NE,1)),np.reshape(xnorm0[NE:,0],(NE,1))
        phie,phii = phie@kseries,phii@kseries
        if transferfun =='tanh':
            derivphie,derivphii = gradtanh(phie),gradtanh(phii)
            muphifunc[0,:]= np.mean(np.tanh(phie),axis=0)[:]
            muphifunc[1,:]= np.mean(np.tanh(phii),axis=0)[:]
            muderivativephifunc[0,:] = np.mean(derivphie,axis=0)[:]
            muderivativephifunc[1,:] = np.mean(derivphii,axis=0)[:]

        ## right hand side of equation, 2 populations\
        rhs = 0
        for ipop in range(2):
            mu_part = (ymu[ipop]*muphifunc[ipop,:])/2.0
            mu_part = np.reshape(mu_part,(1,nknum))
            rhs += mu_part
            cov_part = (xycov[ipop]*(np.reshape(muderivativephifunc[ipop,:],(1,nknum))*kseries)/2.0)
            rhs += cov_part
        rhs = rhs * eigvJ[0]
        nullclines_rhsfun[idxeta,iktrial,:] = rhs[0,:].copy()

        ## Nullclines, intersections
        verrFun = np.abs(rhs-kseries)
        findintersect = np.argsort(verrFun)
        for i in range(nmaxFP*2):
            kappaintersect[idxeta,iktrial,i]=kseries[0,findintersect[0][i]]

        ''' Full Connectivity '''
        xfp0 = np.zeros(2*NE)
        xfp0 = np.random.randn(1,NE*2)*1E-2
        xfp = np.zeros((NE*2,ntt))
        xfp[:,0] = xfp0[:]
        for it in range(1,ntt):
            JPhi = J[:,:]@np.tanh(xfp[:,it-1])
            deltaxfp = -xfp[:,it-1]+JPhi
            xfp[:,it] = deltaxfp*dt + xfp[:,it-1]
        xfpseries[idxeta,iktrial,:,:] = xfp.copy()
        # fig,ax=plt.subplots(2,1,figsize=(4,6))  
        # for i in range(10):     
        #     ax[0].plot(tt,xfp[i,:],linestyle='--',color='red')
        #     ax[1].plot(tt,xfp[NE+i,:],linestyle='--',color='blue')




recording!!!
Total Symmetry: [0. 0. 0. 0. 0. 0.]
compare degree of symmetry: -0.00022480232702854077 --num; 
compare the accuracy: (1.019075433483871+0j) ; test: (1.0190754334838668+6.841974942477513e-28j) ; overlap: (1.0190754334838652+1.5435048363774364e-28j)
overlap, theo: [0.+0.j 0.+0.j] ; num: [0.50108798-7.89712047e-14j 0.36038465+8.11402592e-14j]


<ipython-input-69-c40bfc9e4f75>:294: ComplexWarning: Casting complex values to real discards the imaginary part
  muphifunc[0,:]= np.mean(np.tanh(phie),axis=0)[:]
<ipython-input-69-c40bfc9e4f75>:295: ComplexWarning: Casting complex values to real discards the imaginary part
  muphifunc[1,:]= np.mean(np.tanh(phii),axis=0)[:]
<ipython-input-69-c40bfc9e4f75>:296: ComplexWarning: Casting complex values to real discards the imaginary part
  muderivativephifunc[0,:] = np.mean(derivphie,axis=0)[:]
<ipython-input-69-c40bfc9e4f75>:297: ComplexWarning: Casting complex values to real discards the imaginary part
  muderivativephifunc[1,:] = np.mean(derivphii,axis=0)[:]


Total Symmetry: [0.1 0.1 0.1 0.1 0.1 0.1]
compare degree of symmetry: 0.03176170937399516 --num; 
compare the accuracy: (1.176357914820096+0j) ; test: (1.1763579148200929-3.515360445926162e-28j) ; overlap: (1.1763579148200933+2.969544862556425e-29j)
overlap, theo: [0.1       +0.j 0.94117647+0.j] ; num: [0.36372819+1.49705559e-13j 0.40248137+1.75481764e-14j]
Total Symmetry: [0.2 0.2 0.2 0.2 0.2 0.2]
compare degree of symmetry: 0.06374844335424368 --num; 
compare the accuracy: (1.3013632030809221+0j) ; test: (1.3013632030809192-1.7618426390553877e-28j) ; overlap: (1.3013632030809186-1.6425512870459706e-29j)
overlap, theo: [0.2       +0.j 1.88235294+0.j] ; num: [0.30906127-2.23932549e-13j 0.4023604 +9.39576635e-14j]
Total Symmetry: [0.3 0.3 0.3 0.3 0.3 0.3]
compare degree of symmetry: 0.09573546554579151 --num; 
compare the accuracy: (1.407664923937352+0j) ; test: (1.40766492393735-1.0963428430797255e-28j) ; overlap: (1.4076649239373489-2.6650843827482067e-28j)
overlap, theo: [0.3       +

In [91]:
xticks = np.linspace(-0.3,0.20,2)
xlims = [-0.3,0.20]
yticks = np.linspace(-0.1,0.15,2)
ylims = [-0.1,0.15]
theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''
alphaval=0.50
edgv='black'
cm='br'
fig,ax=plt.subplots(figsize=(5,3))  
idxeta,idxtrial = 2,3

mEvec,mIvec,nEvec,nIvec=np.squeeze(Reigvecseries[idxeta,idxtrial,:NE,0]),np.squeeze(Reigvecseries[idxeta,idxtrial,NE:,0]),np.squeeze(Leigvecseries[idxeta,idxtrial,:NE,0]),np.squeeze(Leigvecseries[idxeta,idxtrial,NE:,0])

ax.scatter(nIvec,mIvec,s=5.0,c='b',alpha=alphaval)#cm[1],alpha=alphaval)
ax.scatter(nEvec,mEvec,s=5.0,c='r',alpha=alphaval)#cm[0],alpha=alphaval)

ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
ax.set_aspect('equal')
print(armu[idxeta,idxtrial,:])



[0.7354076 -0.60296671j 0.64237051-0.5198543j ]
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:138: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order, subok=True)


In [40]:
def scatter_hist(x, y, ax, ax_histx, ax_histy,color,alphaval):
    # no labels
    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)

    # the scatter plot:
    ax.scatter(x, y, s=5.0,c=color ,alpha=alphaval)#cm[0],alpha=alphaval)

    # now determine nice limits by hand:
    binwidth = 0.05
    xymax = max(np.max(np.abs(x)), np.max(np.abs(y)))
    lim = (int(xymax/binwidth) + 1) * binwidth
    binsx = np.arange(-lim, lim + binwidth, binwidth)

    binwidth = 0.01
    xymax = max(np.max(np.abs(x)), np.max(np.abs(y)))
    lim = (int(xymax/binwidth) + 1) * binwidth
    binsy = np.arange(-lim, lim + binwidth, binwidth)

    
    ax_histx.hist(x, bins=binsx,density=True,color=color,alpha=alphaval)
    ax_histy.hist(y, bins=binsy, orientation='horizontal',density=True,color=color,alpha=alphaval)

In [181]:
'''
loading patterns
for ensemble-averaging results
>>>>>>>>>>>>>>>>> with eigenvalues >>>>>>>>>>>>>>>
'''
# xticks = np.linspace(-0.2,0.25,2)
# xlims = [-0.2,0.25]
# yticks = np.linspace(-0.2,0.25,2)
# ylims = [-0.2,0.25]
# xticks = np.linspace(-0.15,0.10,2)
# xlims = [-0.15,0.10]
# yticks = np.linspace(-0.06,0.01,2)
# ylims = [-0.06,0.01]
# xticks = np.linspace(-0.3,0.3,2)
# xlims = [-0.3,0.3]
# yticks = np.linspace(-0.25,0.25,2)
# ylims = [-0.25,0.25]
theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''

idxtrial=2
idxeta=int(neta/2)
alphaval=0.10
edgv='blue'#black'
cm='br'
fig,ax=plt.subplots(figsize=(5,3)) 
# # start with a square Figure
# fig = plt.figure(figsize=(5, 3)) 
nrank=1
ntrial = np.shape(Reigvecseries)[1]
NE=int(NE)
print(NE)
'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)

mEvec,mIvec,nEvec,nIvec=np.squeeze(Reigvecseries[idxeta,:,:NE,0]),np.squeeze(Reigvecseries[idxeta,:,NE:,0]),np.squeeze(Leigvecseries[idxeta,:,:NE,0]),np.squeeze(Leigvecseries[idxeta,:,NE:,0])
mEvec,mIvec,nEvec,nIvec=mEvec.flatten()/np.sqrt(NE*2),mIvec.flatten()/np.sqrt(NE*2),nEvec.flatten()/np.sqrt(NE*2),nIvec.flatten()/np.sqrt(NE*2)
scale_std=3.0
for irank in range(nrank):
    meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
    print('meanE:',)
    varmE = mEvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEvec[idxwhere]=meanmEtotal
    meanmE[irank]=np.mean(mEvec)

    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
    varnE = nEvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEvec[idxwhere]=meannEtotal
    meannE[irank]=np.mean(nEvec)

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

''' calculate directions '''
noiseE,noiseI = np.zeros((NE*ntrial,2,nrank)),np.zeros((NE*ntrial,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)  

xticks = np.linspace(-0.5,0.5,2)
xlims = [-0.5,0.5]
yticks = np.linspace(-0.15,0.15,2)
ylims = [-0.1,0.1]
# definitions for the axes
left, width = 0.1, 0.65
bottom, height = 0.1, 0.35
spacing = 0.005

rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom + height + spacing, width, 0.2]
rect_histy = [left + width + spacing, bottom, 0.2, height]

# ax.scatter(nIvec,mIvec,s=5.0,c='b',alpha=alphaval)#cm[1],alpha=alphaval)
# ax.scatter(nEvec,mEvec,s=5.0,c='r',alpha=alphaval)#cm[0],alpha=alphaval)

''' integrating hist and scatter '''
# ax = fig.add_axes(rect_scatter)
# ax_histx = fig.add_axes(rect_histx, sharex=ax)
# ax_histy = fig.add_axes(rect_histy, sharey=ax)
# # use the previously defined function
# scatter_hist(nEvec,mEvec , ax, ax_histx, ax_histy,cm[1],alphaval)
# scatter_hist(nIvec,mIvec , ax, ax_histx, ax_histy,cm[0],alphaval)

ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color=cm[1],linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color=cm[1],linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color=cm[0],linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color=cm[0],linestyle='--',linewidth=1.5)

confidence_ellipse(np.real(nEvec),np.real(mEvec),ax,edgecolor=cm[1])#edgv)
confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=cm[0])
ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')

abinsx = np.linspace(-0.5,0.5,50)
abinsy = np.linspace(-0.1,0.1,50)
fig,axxhist=plt.subplots(figsize=(5,1.5)) 
n, bins, patches=axxhist.hist(nEvec, bins=abinsx,density=True,color=cm[1],alpha=alphaval)
xe = ((1 / (np.sqrt(2 * np.pi) * stdnEtotal))*np.exp(-0.5 * (1 / stdnEtotal * (bins - meannE[0]))**2))
axxhist.plot(bins, xe, '--',c=cm[1])
n, bins, patches=axxhist.hist(nIvec, bins=abinsx,density=True,color=cm[0],alpha=alphaval)
xi = ((1 / (np.sqrt(2 * np.pi) * stdnItotal))*np.exp(-0.5 * (1 / stdnItotal * (bins - meannI[0]))**2))
axxhist.plot(bins, xi, '--',c=cm[0])
fig,ayyhist=plt.subplots(figsize=(2,1.5)) 
n, bins, patches=ayyhist.hist(mEvec, bins=abinsy,density=True,color=cm[1],alpha=alphaval)
ye = ((1 / (np.sqrt(2 * np.pi) * stdmEtotal))*np.exp(-0.5 * (1 / stdmEtotal * (bins - meanmE[0]))**2))
ayyhist.plot(bins, ye, '--',c=cm[1])
n, bins, patches=ayyhist.hist(mIvec, bins=abinsy,density=True,color=cm[0],alpha=alphaval)
yi = ((1 / (np.sqrt(2 * np.pi) * stdmItotal))*np.exp(-0.5 * (1 / stdmItotal * (bins - meanmI[0]))**2))
ayyhist.plot(bins, yi, '--',c=cm[0])


750
meanE:
(2, 2)
(2, 2)


<ipython-input-181-61ccdb93cf1b>:45: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmE[irank]=np.mean(mEvec)
<ipython-input-181-61ccdb93cf1b>:52: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmI[irank]=np.mean(mIvec)
<ipython-input-181-61ccdb93cf1b>:60: ComplexWarning: Casting complex values to real discards the imaginary part
  meannE[irank]=np.mean(nEvec)
<ipython-input-181-61ccdb93cf1b>:67: ComplexWarning: Casting complex values to real discards the imaginary part
  meannI[irank]=np.mean(nIvec)
<ipython-input-181-61ccdb93cf1b>:74: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
<ipython-input-181-61ccdb93cf1b>:75: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]


In [189]:
xticks = np.linspace(-2.0,2.0,2)
xlims = [-2.0,2.0]
yticks = np.linspace(-2.0,2.0,2)
ylims = [-2.0,2.0]

idxeta,iktrial = 2,2
abinsx = np.linspace(-2.,2.,50)
# ## -------- different realizations -------
# for idxeta in range(int(neta/2)):
#     fig,ax = plt.subplots(figsize=(4,4))
#     ax.plot(kseries[0,:],kseries[0,:],color='gray',linestyle='--')
#     for iktrial in range(1,ntrial):
#         ax.plot(kseries[0,:],nullclines_rhsfun[idxeta,iktrial,:],color='black',linewidth=1.5,alpha=0.5)
#     ax.set_xlim(xlims)
#     ax.set_ylim(ylims)
#     ax.set_xticks(xticks)
#     ax.set_yticks(yticks)
#     ax.set_aspect('equal')
#     stt = np.zeros((NE*ntrial,2))
#     stt[:,0],stt[:,1] = xfpseries[idxeta,:,:NE,-1].flatten(),xfpseries[idxeta,:,NE:,-1].flatten()
#     fig,ax=plt.subplots(2,1,figsize=(4,4))  
#     ax[0].hist(stt[:,0], bins=n_bins)
#     ax[1].hist(stt[:,1], bins=n_bins)

## ------------ single trial --------------
for idxeta in range(int(neta/2)):
    fig,ax = plt.subplots(figsize=(4,4))
    ax.plot(kseries[0,:],kseries[0,:],color='gray',linestyle='--')
    ax.plot(kseries[0,:],np.real(nullclines_rhsfun[idxeta,iktrial,:]),color='black',linewidth=1.5,alpha=0.5)
    if idxeta <1:
        ax.plot(kseries[0,:],np.real(kappaAmfunc),color='blue',linestyle='-.',linewidth=1.5,alpha=0.5)
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)
    ax.set_aspect('equal')
    stt = np.zeros((NE*ntrial,2))
    stt[:,0],stt[:,1] = np.real(xfpseries[idxeta,:,:NE,-1]).flatten(),np.real(xfpseries[idxeta,:,NE:,-1]).flatten()
    fig,ax=plt.subplots(2,1,figsize=(4,4))  
    ax[0].hist(stt[:,0], bins=abinsx,density=True,color=cm[1],alpha=alphaval*5)
    ax[1].hist(stt[:,1], bins=abinsx,density=True,color=cm[0],alpha=alphaval*5)

In [196]:
idxeta,iktrial = 2,2

# # fig,ax=plt.subplots(2,1,figsize=(4,6)) 
# xticks = np.linspace(0,100,2) 
# xlims = [0.,100]
# yticks = np.linspace(-1.5,1.5,2)
# ylims = [-0.025,0.025]
# for idxeta in range(int(neta/2)):
#     fig,ax=plt.subplots(2,1,figsize=(4,4))
#     for i in range(0,NE,50):
#         ax[0].plot(tt[:1000],np.real(xfpseries[idxeta,iktrial,i,:1000]),color='red')
#         ax[1].plot(tt[:1000],np.real(xfpseries[idxeta,iktrial,NE+i,:1000]),color='blue')
#         xticks = np.linspace(-1.,1.5,2)
#     for i in range(2):
#         # ax[i].set_xlim(xlims)
#         ax[i].set_ylim(ylims)
#         # ax[i].set_xticks(xticks)
#         ax[i].set_yticks(yticks)

xticks = np.linspace(0,100,2) 
xlims = [0.,100]
yticks = np.linspace(-1.5,1.5,2)
ylims = [-1.5,1.5]
idxeta = 2
ftime = 1000
fig,ax=plt.subplots(2,1,figsize=(4,4))
for i in range(100,NE,100):
    ax[0].plot(tt[:ftime],np.real(xfpseries[idxeta,iktrial,i,:ftime]),color='red')
    ax[1].plot(tt[:ftime],np.real(xfpseries[idxeta,iktrial,NE+i,:ftime]),color='blue')
    xticks = np.linspace(-1.,1.5,2)
for i in range(2):
    # ax[i].set_xlim(xlims)
    ax[i].set_ylim(ylims)
    # ax[i].set_xticks(xticks)
    ax[i].set_yticks(yticks)

In [185]:

'''
with symmetry, subcircuit
eigvJ0
'''

# ## ------------ different realizations ---------
# idxc=0
# print(ntrial)
# idxgavg,idxtrial=9,8 # 6,3,0
# for idxgavg in range(int(neta/2)):
#     figtspt,axtspt=plt.subplots(figsize=(6,3))
#     for idxtrial in range(ntrial):
#         axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,:]),np.imag(Beigvseries[idxgavg,idxtrial,:]),s=5,c=cm[0],alpha=0.25) # >>>>>>>>>>>>>>
#     axtspt.scatter(np.real(Beigvseries[idxgavg,:,0]),np.imag(Beigvseries[idxgavg,:,0])-0.2*idxc,s=20,c='red',marker='^',alpha=0.5) # >>>>>>>>>>>>
#     axtspt.scatter(np.real(Beigvseries[idxgavg,:,1]),np.imag(Beigvseries[idxgavg,:,1])-0.2*idxc,s=20,c='darkblue',marker='^',alpha=0.5) # >>>>>>>>>>>>
#     axtspt.set_aspect('equal')
#     axtspt.plot(np.ones(2)*eigvAm[0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
#     axtspt.plot(np.ones(2),np.array([-1.0,1.0]),linestyle='-.',linewidth=1.5,color='black')

#     axtspt.spines['right'].set_color('none')
#     axtspt.spines['top'].set_color('none')
#     axtspt.xaxis.set_ticks_position('bottom')
#     axtspt.spines['bottom'].set_position(('data', 0))
#     xticks = np.linspace(-1.5,1.5,2)
#     xlims = [-1.5,1.5]
#     yticks = np.linspace(-1.0,1.0,2)
#     ylims = [-1.0,1.0]
#     axtspt.set_xlim(xlims)
#     axtspt.set_ylim(ylims)
#     axtspt.set_xticks(xticks)
#     axtspt.set_yticks(yticks)
#     axtspt.set_aspect('equal')

##n ---------- single trial ----------
idxc=0
print(ntrial)
idxgavg,idxtrial=9,2 # 6,3,0
for idxgavg in range(int(neta/2)):
    figtspt,axtspt=plt.subplots(figsize=(6,3))
    axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,:]),np.imag(Beigvseries[idxgavg,idxtrial,:]),s=5,c=cm[0],alpha=0.25) # >>>>>>>>>>>>>>
    axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,0]),np.imag(Beigvseries[idxgavg,idxtrial,0])-0.2*idxc,s=20,c='red',marker='^',alpha=0.5) # >>>>>>>>>>>>
    axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,1]),np.imag(Beigvseries[idxgavg,idxtrial,1])-0.2*idxc,s=20,c='darkblue',marker='^',alpha=0.5) # >>>>>>>>>>>>
    axtspt.set_aspect('equal')
    axtspt.plot(np.ones(2)*np.real(eigvAm[0]),np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
    axtspt.plot(np.ones(2),np.array([-1.0,1.0]),linestyle='-.',linewidth=1.5,color='black')

    axtspt.spines['right'].set_color('none')
    axtspt.spines['top'].set_color('none')
    axtspt.xaxis.set_ticks_position('bottom')
    axtspt.spines['bottom'].set_position(('data', 0))
    xticks = np.linspace(-1.,2.0,2)
    xlims = [-1.,2.0]
    yticks = np.linspace(-0.8,0.8,2)
    ylims = [-0.8,0.8]
    axtspt.set_xlim(xlims)
    axtspt.set_ylim(ylims)
    axtspt.set_xticks(xticks)
    axtspt.set_yticks(yticks)
    axtspt.set_aspect('equal')

# # iktrial=0
# # fig,ax=plt.subplots(figsize=(4,4))  
# # ax.plot(kseries[0,:],kseries[0,:],linestyle='--',color='grey')
# # ax.plot(kseries[0,:],rhsfun[iktrial,:],color='r',linewidth=1.5)
# idxeta,idxtrial = 0,3
# xticks = np.linspace(-2.0,2.0,2)
# xlims = [-2.0,2.0]
# yticks = np.linspace(-1.0,1.0,2)
# ylims = [-1.0,1.0]

# fig,ax=plt.subplots(figsize=(4,4))  
# # ax.set_aspect('equal')
# ax.scatter(np.real(Beigvseries[idxeta,idxtrial,:]),np.imag(Beigvseries[idxeta,idxtrial,:]),s=5,color='grey')
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_aspect('equal')


3


## Multiple Populations (heterogeneous - 3)

In [33]:
'''
WITH SYMMETRY
'''
'''
* multi-population (example 2 E and 1 I)
* homogeneous random gain g
* heterogeneous degree of symmetry
* two distinguished E populations
'''
# generate mean matrix
nrank=1
Nt=np.array([750,750])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
nrank,ntrial,neta,nvec,nmaxFP=1,3,11,2,3
npop = 3

## for all casesq
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)

''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

''' ---------- method 2 ------------------ '''
''' ## Three \bar{J} cases '''
JI,JE,a,b=0.575+1.0,1.325+1.0,0,0#0.575+1.0,1.325+1.75,0,0#
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
''' for reference, eigenvectors of unperturbed matrix \bar{J}'''
xAm,yAm=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))

mAm,nAm = xAm.copy()*np.sqrt(N),yAm.copy()*np.sqrt(N)*eigvAm[0]
phiAm = mAm@kseries
muphiAmE,muphiAmI = np.mean(np.tanh(phiAm[:NE,:]),axis=0),np.mean(np.tanh(phiAm[NE:,:]),axis=0)
kappaAmfunc = np.zeros(nknum)
kappaAmfunc = np.mean(nAm[:NE,0])*muphiAmE/2.0+np.mean(nAm[NE:,0])*muphiAmI/2.0

''' reference '''
l0,r0 = np.zeros(3),np.ones(3)
l0[0],l0[1],l0[2] = JE*2/(JE-JI),JE*2/(JE-JI),-JI*2/(JE-JI) # mnT/N -- so *sqrt(N) already
''' Network Setting  '''
## heterogeneous random gain
gaverage = 0.6
xee,xei,xie,xii=1.0,1.0,1.0,1.0 

## heterogeneous degree of symmetry: amplitudes and signs
coeffetaEsub=np.array([1.0,1.0,1.0])#
coeffetaTotal = np.array([1.0,1.0,1.0])#
# coeffetaTotal = np.zeros(3)
signetaEsub=np.ones(3)
signetaTotal=np.ones(3)
signetaTotal[2]=-1
# signetaTotal[:]=-1
# signetaTotal[1]=+1

ppercentEsub = np.ones(2)
ppercentEsub[0]=0.5
ppercentEsub[1]=1.0-ppercentEsub[0]
## E->total ppercentEsub[0]/2.0,ppercentEsub[1]/2.0, (I) 1/2.0

''' switch to 3 populations '''
ppercentTotal = np.zeros(npop)
ppercentTotal[:2] = ppercentEsub[:]/2.0
ppercentTotal[2] = 1/2.0
## check
sum_percent = np.sum(ppercentTotal)
if sum_percent!=1:
    print('Probabilities do not add up to one, ',sum_percent)

'''  Recording Variables. '''
## perturbed + symmetric eigenvectors
Reigvecseries,Leigvecseries=np.zeros((neta,ntrial,NE*2,nvec*2),dtype=complex),np.zeros((neta,ntrial,NE*2,nvec*2),dtype=complex)
Beigvseries=np.zeros((neta,ntrial,N),dtype=complex)
## corresponding statistics \mu and \sigma \cov
armu,sigrcov = np.zeros((neta,ntrial,npop),dtype=complex),np.zeros((neta,ntrial,npop),dtype=complex) # 2 for E and I
almu,siglcov = np.zeros((neta,ntrial,npop),dtype=complex),np.zeros((neta,ntrial,npop),dtype=complex)
sigrl = np.zeros((neta,ntrial,npop),dtype=complex)
## reference, eigenvectors of matrix with iid perturbations
x0series,y0series=np.zeros((ntrial,NE*2)),np.zeros((ntrial,NE*2))

''' Nullclines parameters  '''
nknum = 81
kseries = np.reshape(np.linspace(-4.,4.,nknum),(1,nknum))
transferfun = 'tanh'

nullclines_rhsfun = np.zeros((neta,ntrial,nknum),dtype=complex)
kappaintersect    = np.zeros((neta,ntrial,nmaxFP*2))

''' simulating parameters '''
tt = np.linspace(0,200,2000)
dt = tt[2]-tt[1]
ntt = len(tt)
xfpseries = np.zeros((neta,ntrial,NE*2,ntt),dtype=complex)
''' All random matrices for each trials '''
## Not X total (symmetry), nut Y independent
gYbasexee=np.zeros((ntrial,NE*2,NE*2),dtype=complex)
''' Iterative Processing '''
etaseries = np.linspace(0.0,1.0,neta)
# if eta == 0.0:
for iktrial in range(ntrial):
    '''    ##>>>>>>>>>>> g0!=0, >>>>>>>>     '''
    Xsym=np.random.randn(NE*2,NE*2)*gaverage/np.sqrt(2*NE) # /np.sqrt(2) no later
    Xsym[:NE,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[:NE,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[NE:,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[NE:,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    XsymT=Xsym.copy().T
    X0=Xsym.copy()
    J0=Am.copy()+X0.copy()
    ''' # calculate the original \bar{m} and \bar{n} '''
    eigvJ0,eigrvecJ0=la.eig(J0)
    inveigrvecJ0=la.inv(eigrvecJ0)
    meig0 = np.squeeze(eigrvecJ0[:,:].copy())
    neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
    neig0=neig0.copy().T
    for i in range(nrank):
        neig0[:,i]*=eigvJ0[i]
    for j in range(nrank,N):
        neig0[:,j]*=eigvJ0[j]

    leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
    ''' The first '''
    reig0=meig0.copy()
    normval=np.sum(reig0*neig0.copy(),axis=0)
    normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
    leig0=neig0.copy()/normval.copy()
    ## for reference
    tildex,tildey=np.reshape(reig0[:,0].copy(),(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
    ## make sure the E in y is positive, for negative change signs of x and y
    if (np.mean(tildey[:NE,0])*yAm[0,0])<0:
        tildex*=(-1)
        tildey*=(-1)
    x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
    x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
    y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey

    x0series[iktrial,:],y0series[iktrial,:]=x0[:,0].copy(),y0[:,0].copy()
    ''' record the random derivative '''
    print('recording!!!')
    ## log: there is a bug, X is the total random xconn, including symmetry
    gYbasexee[iktrial,:,:]=Xsym.copy()
    for idxeta, eta in enumerate(etaseries[::2]):
        etaset=eta*np.ones(6)#np.zeros(3)#eta*np.ones(3)
        for iloop in range(3):
            etaset[iloop] = coeffetaEsub[iloop]*etaset[iloop]
            etaset[iloop+3] = coeffetaTotal[iloop]*etaset[iloop+3]
        print('Total Symmetry:',etaset)
        Xinit = Xsym.copy()
        ### >>>>>>>>.Subcircuit Exc sym >>>>>>>>>.
        ## remaining
        hNE=int(ppercentEsub[0]*NE) 
        asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[:hNE,:hNE]=signetaEsub[0]*aamp*XsymT[:hNE,:hNE].copy()+np.sqrt(1-aamp**2)*Xsym[:hNE,:hNE].copy()
        ## >>>>>>. esub middle
        asqr=(1-np.sqrt(1-etaset[1]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[:hNE,hNE:NE]=signetaEsub[1]*aamp*XsymT[:hNE,hNE:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:hNE,hNE:NE].copy()
        Xinit[hNE:NE,:hNE]=signetaEsub[1]*aamp*XsymT[hNE:NE,:hNE].copy()+np.sqrt(1-aamp**2)*Xsym[hNE:NE,:hNE].copy()
        ## >>>>>> E2
        asqr=(1-np.sqrt(1-etaset[2]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[hNE:NE,hNE:NE]=signetaEsub[2]*aamp*XsymT[hNE:NE,hNE:NE].copy()+np.sqrt(1-aamp**2)*Xsym[hNE:NE,hNE:NE].copy()
        ### >>>>>>> Total >>>>>
        # E1I-B
        hNE = int(ppercentEsub[0]*NE) 
        asqr=(1-np.sqrt(1-(etaset[3])**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[NE:,:hNE]=signetaTotal[0]*aamp*XsymT[NE:,:hNE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:hNE].copy()
        Xinit[:hNE,NE:]=signetaTotal[0]*aamp*XsymT[:hNE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[:hNE,NE:].copy()
        # E2I-B
        asqr=(1-np.sqrt(1-(etaset[4])**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[NE:,hNE:NE]=signetaTotal[1]*aamp*XsymT[NE:,hNE:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,hNE:NE].copy()
        Xinit[hNE:NE,NE:]=signetaTotal[1]*aamp*XsymT[hNE:NE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[hNE:NE,NE:].copy()
        ## II ##
        asqr=(1-np.sqrt(1-etaset[5]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,NE:]=signetaTotal[2]*aamp*XsymT[NE:,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,NE:].copy()
                    
        X=Xinit.copy()
        X[:NE,:NE]*=(xee)
        X[:NE,NE:]*=(xei)
        X[NE:,:NE]*=(xie)
        X[NE:,NE:]*=(xii)

        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
        symnumfull*=2*NE
        print('compare degree of symmetry:',symnumfull,'--num; ')#,etaset*signeta,'--setting')
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        # meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
        for j in range(nrank,N):
            neig[:,j]*=eigvJ[j]
        '''    Sort Eigenvalue in ascending     '''
        eigenvalue_Rsort = np.squeeze(-np.abs(np.real(eigvJ.copy())))
        idxsort=np.argsort(eigenvalue_Rsort)
        eigenvalue_sort = np.squeeze(eigvJ.copy())
        eigvJ= eigvJ[idxsort]  #### >>>>>>>>>>>>>> resorting >>>>>>>>>
        ## >>> for eigendecomposition 
        Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
        Reig = meig[:,idxsort]
        Leig = neig[:,idxsort]
        normval=np.sum(Reig*Leig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        Leig=Leig.copy()/normval.copy()   # error
        # print('test normal 5:',np.sum(Leig[:,0]*Reig[:,0]))
        '''   normalization 0 (reference)   '''
        for irec in range(1):
            tildex,tildey=np.reshape(Reig[:,irec].copy(),(2*NE,1)),np.reshape(Leig[:,irec].copy(),(NE*2,1))
            if (np.mean(tildey[:NE,0])*yAm[0,0])<0:
                tildex*=(-1)
                tildey*=(-1)
            xnorm0=np.sqrt(np.squeeze(tildey.T@x0)/np.squeeze(y0.T@tildex))*tildex
            ynorm0=np.sqrt(np.squeeze(tildex.T@y0)/np.squeeze(x0.T@tildey))*tildey

            '''
            ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N 
            '''
            xnorm0,ynorm0 = xnorm0*np.sqrt(2*NE),ynorm0*np.sqrt(2*NE)
            Reigvecseries[idxeta,iktrial,:,irec],Leigvecseries[idxeta,iktrial,:,irec]=xnorm0[:,0].copy(),ynorm0[:,0].copy()
            ''' calculate statistical properties of THREE popoulations ''' 
            ## statistics mean(mu) and variance(std/var)
            idxstart=0
            for ipop in range(npop):
                numdur = ppercentTotal[ipop]*(NE*2)
                numdur = int(numdur)
                idxend = idxstart+numdur
                armu[idxeta,iktrial,ipop] = np.mean(xnorm0[idxstart:idxend,irec])
                sigrcov[idxeta,iktrial,ipop] = np.std(xnorm0[idxstart:idxend,irec])**2
                almu[idxeta,iktrial,ipop] = np.mean(ynorm0[idxstart:idxend,irec])
                siglcov[idxeta,iktrial,ipop] = np.std(ynorm0[idxstart:idxend,irec])**2
                rlcov = (xnorm0[idxstart:idxend,irec]-np.mean(xnorm0[idxstart:idxend,irec]))*(ynorm0[idxstart:idxend,irec]-np.mean(ynorm0[idxstart:idxend,irec]))
                sigrl[idxeta,iktrial,ipop] = np.mean(rlcov)
                idxstart = idxend ## set the new start point
                # print('mean:',armu[idxeta,iktrial,ipop])



            # armu[idxeta,iktrial,0],armu[idxeta,iktrial,1] = np.mean(xnorm0[:NE,0]),np.mean(xnorm0[NE:,0])
            # sigrcov[idxeta,iktrial,0],sigrcov[idxeta,iktrial,1] = np.std(xnorm0[:NE,0])**2,np.std(xnorm0[NE:,0])**2
            # testsigrcove,testsigrcovi = np.sum((Reigvecseries[idxeta,iktrial,:NE,irec]-armu[idxeta,iktrial,0])**2)/NE,np.sum((Reigvecseries[idxeta,iktrial,NE:,irec]-armu[idxeta,iktrial,1])**2)/NE
            # print('compare single sigma:',sigrcov[idxeta,iktrial,:],'; testmanu:',testsigrcove,testsigrcovi)
            # almu[idxeta,iktrial,0],almu[idxeta,iktrial,1] =np.mean(ynorm0[:NE,0]),np.mean(ynorm0[NE:,0])
            # siglcov[idxeta,iktrial,0],siglcov[idxeta,iktrial,1] = np.std(ynorm0[:NE,0])**2,np.std(ynorm0[NE:,0])**2
            # rlcove = (xnorm0[:NE,0]-np.mean(xnorm0[:NE,0]))*(ynorm0[:NE,0]-np.mean(ynorm0[:NE,0]))
            # print('shape !!!!:',np.shape(rlcove))
            # sigrl[idxeta,iktrial,0] = np.sum(rlcove)/NE
            # rlcovi = (xnorm0[NE:,0]-np.mean(xnorm0[NE:,0]))*(ynorm0[NE:,0]-np.mean(ynorm0[NE:,0]))
            # sigrl[idxeta,iktrial,1] = np.sum(rlcovi)/NE

            # print('compare e:',np.mean(rlcove),';', sigrl[idxeta,iktrial,0])
            # print('compare i:',np.mean(rlcovi),';', sigrl[idxeta,iktrial,1])
        
        ''' test reconstruction '''
        rtest,ltest = np.squeeze(Reigvecseries[idxeta,iktrial,:,0].copy()),np.squeeze(Leigvecseries[idxeta,iktrial,:,0].copy())
        rtest,ltest = np.reshape(np.squeeze(rtest),(2*NE,1)),np.reshape(np.squeeze(ltest),(2*NE,1))
        Jtest = rtest@ltest.T/NE/2*eigvJ[0]
        eigvtest,_ = la.eig(Jtest)
        
        ''' overlap mat's eigenvalue should be the same '''
        Jov = 0
        for ipop in range(npop):
            mu_part = armu[idxeta,iktrial,ipop]*almu[idxeta,iktrial,ipop]
            var_part = sigrl[idxeta,iktrial,ipop]
            Jov += ppercentTotal[ipop]*eigvJ[0]*(mu_part+var_part)
        print('compare the accuracy:',eigvJ[0],'; test:',eigvtest[0],'; overlap:',Jov)
        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()

        ## Rank One Approximation, Nullclines of \kappa
        xnorm0, ynorm0 = Reigvecseries[idxeta,iktrial,:,0].copy(),Leigvecseries[idxeta,iktrial,:,0]
        xnorm0, ynorm0 = np.reshape(xnorm0,(2*NE,1)),np.reshape(ynorm0,(2*NE,1))
        xmu,ymu = armu[idxeta,iktrial,:].copy(),almu[idxeta,iktrial,:].copy()
        xsig,ysig = sigrcov[idxeta,iktrial,:].copy(),siglcov[idxeta,iktrial,:].copy()
        xycov = sigrl[idxeta,iktrial,:].copy()
        ''' kappa dynamics    '''
        overlape=gaverage**2/eigvAm[0]**2*eta
        
        overlaptheo = np.zeros(3,dtype=complex)
        ''' calculate theoretical Jov '''
        # E1
        ''' there is an error, without gaverage**2*eigvJ0[0]/eigvAm[0] ''' 
        overlaptheo[0]=(l0[0]*r0[0]*etaset[0]*signetaEsub[0]*ppercentTotal[0]+l0[1]*r0[1]*etaset[1]*signetaEsub[1]*ppercentTotal[1]+l0[2]*r0[2]*etaset[3]*signetaTotal[0]*ppercentTotal[2])#*eigvJ0[0]*gaverage**2
        overlaptheo[1]=(l0[0]*r0[0]*etaset[1]*signetaEsub[1]*ppercentTotal[0]+l0[1]*r0[1]*etaset[2]*signetaEsub[2]*ppercentTotal[1]+l0[2]*r0[2]*etaset[4]*signetaTotal[1]*ppercentTotal[2])#*eigvJ0[0]*gaverage**2
        overlaptheo[2]=(l0[0]*r0[0]*etaset[3]*signetaTotal[0]*ppercentTotal[0]+l0[1]*r0[1]*etaset[4]*signetaTotal[1]*ppercentTotal[1]+l0[2]*r0[2]*etaset[5]*signetaTotal[2]*ppercentTotal[2])#*eigvJ0[0]*gaverage**2
        overlaptheo *=(gaverage**2/eigvAm[0]**2)
        ## >>>>>>> here is the coefficient, but doesn't have eigvJ0[0], should be equal
        print('overlap, theo',overlaptheo,'; num:',xycov)
        ## 3 populations 
        ## phie and phii, mu phie mu phii, derivative phie phii, mu derivative phie phii
        muphifunc,muderivativephifunc = np.zeros((npop,nknum),dtype=complex),np.zeros((npop,nknum),dtype=complex)
        idxstart=0
        for ipop in range(npop):
            numdur = ppercentTotal[ipop]*(NE*2)
            numdur = int(numdur)
            idxend = idxstart+numdur
            phix = np.reshape(xnorm0[idxstart:idxend,0],(numdur,1))
            phix = phix@kseries

            derivphix = gradtanh(phix)
            muphifunc[ipop,:] = np.mean(np.tanh(phix),axis=0)[:]
            muderivativephifunc[ipop,:] = np.mean(derivphix,axis=0)[:]

            idxstart = idxend
        # phie, phii = np.reshape(xnorm0[:NE,0],(NE,1)),np.reshape(xnorm0[NE:,0],(NE,1))
        # phie,phii = phie@kseries,phii@kseries
        # if transferfun =='tanh':
        #     derivphie,derivphii = gradtanh(phie),gradtanh(phii)
        #     muphifunc[0,:]= np.mean(np.tanh(phie),axis=0)[:]
        #     muphifunc[1,:]= np.mean(np.tanh(phii),axis=0)[:]
        #     muderivativephifunc[0,:] = np.mean(derivphie,axis=0)[:]
        #     muderivativephifunc[1,:] = np.mean(derivphii,axis=0)[:]

        ## right hand side of equation, 2 populations\
        rhs = 0
        for ipop in range(npop):
            mu_part = (ymu[ipop]*muphifunc[ipop,:])*ppercentTotal[ipop]
            mu_part = np.reshape(mu_part,(1,nknum))
            rhs += mu_part
            ''' an error here '''
            cov_part = (xycov[ipop]*(np.reshape(muderivativephifunc[ipop,:],(1,nknum))*kseries))*ppercentTotal[ipop]
            rhs += cov_part
        rhs = rhs * eigvJ[0] # HERE EIGVJ0[0]
        nullclines_rhsfun[idxeta,iktrial,:] = rhs[0,:].copy()

        ## Nullclines, intersections
        verrFun = np.abs(rhs-kseries)
        findintersect = np.argsort(verrFun)
        for i in range(nmaxFP*2):
            kappaintersect[idxeta,iktrial,i]=kseries[0,findintersect[0][i]]

        ''' Full Connectivity '''
        xfp0 = np.zeros(2*NE)
        xfp0 = kappaintersect[idxeta,iktrial,0]*xnorm0[:,0]+np.random.randn(1,NE*2)*1E-2
        xfp = np.zeros((NE*2,ntt),dtype=complex)
        xfp[:,0] = xfp0[:]
        for it in range(1,ntt):
            JPhi = J[:,:]@np.tanh(xfp[:,it-1])
            deltaxfp = -xfp[:,it-1]+JPhi
            xfp[:,it] = deltaxfp*dt + xfp[:,it-1]
        xfpseries[idxeta,iktrial,:,:] = xfp.copy()
        # fig,ax=plt.subplots(2,1,figsize=(4,6))  
        # for i in range(10):     
        #     ax[0].plot(tt,xfp[i,:],linestyle='--',color='red')
        #     ax[1].plot(tt,xfp[NE+i,:],linestyle='--',color='blue')




<ipython-input-33-2ab2b7ffe702>:53: RuntimeWarning: divide by zero encountered in true_divide
  neigvecAm[:,i]/=normalizecoe
<ipython-input-33-2ab2b7ffe702>:53: RuntimeWarning: invalid value encountered in true_divide
  neigvecAm[:,i]/=normalizecoe
<ipython-input-33-2ab2b7ffe702>:162: ComplexWarning: Casting complex values to real discards the imaginary part
  x0series[iktrial,:],y0series[iktrial,:]=x0[:,0].copy(),y0[:,0].copy()


recording!!!
Total Symmetry: [0. 0. 0. 0. 0. 0.]
compare degree of symmetry: 0.0002671101326335601 --num; 
compare the accuracy: (0.6717343404909918+0j) ; test: (0.6717343404909923+1.564877397010752e-17j) ; overlap: (0.6717343404909925-6.326061344630616e-17j)
overlap, theo [0.+0.j 0.+0.j 0.+0.j] ; num: [ 0.17519084-8.40858748e-14j -0.00720629-6.99698725e-14j
  1.08446262-1.43262191e-13j]
Total Symmetry: [0.2 0.2 0.2 0.2 0.2 0.2]
compare degree of symmetry: 0.03631487600550079 --num; 
compare the accuracy: (0.8900392348210413+0j) ; test: (0.8900392348210442-4.806063736846811e-18j) ; overlap: (0.8900392348210433+3.885517780342005e-17j)
overlap, theo [0.128 +0.j 0.128 +0.j 0.6656+0.j] ; num: [-0.23849063+6.08390165e-15j -0.05208576+1.70433842e-13j
  0.30539326+3.90322425e-14j]
Total Symmetry: [0.4 0.4 0.4 0.4 0.4 0.4]
compare degree of symmetry: 0.07236251343068281 --num; 
compare the accuracy: (1.0413936697945199+0j) ; test: (1.041393669794516+1.2724920683712873e-17j) ; overlap: (1.04139

KeyboardInterrupt: 

In [122]:
mAm,nAm = xAm.copy()*np.sqrt(N),yAm.copy()*np.sqrt(N)*eigvAm[0]
phiAm = mAm@kseries
muphiAmE,muphiAmI = np.mean(np.tanh(phiAm[:NE,:]),axis=0),np.mean(np.tanh(phiAm[NE:,:]),axis=0)
kappaAmfunc = np.zeros(nknum)
kappaAmfunc = np.mean(nAm[:NE,0])*muphiAmE/2.0+np.mean(nAm[NE:,0])*muphiAmI/2.0

## flexible numbers

In [111]:
N = 1000
pN = np.ones(4)/4.0
JAm = np.zeros((N,N))
JI,JE=0.575,1.325
Nunit = int(pN[0]*N)
JAm[:,:3*Nunit],JAm[:,3*Nunit:] = JE*4.0/3.0/N,-JI*4.0/1.0/N
vAm,vecAm = la.eig(JAm)
print(vAm[0],JE-JI)

(0.749999999999997+0j) 0.75


## General Conditions

In [26]:
# generate mean matrix
nrank=1
Nt=np.array([750,750])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
nrank,ntrial,neta,nvec,nmaxFP=1,3,11,2,3
npop = 3

nknum = 41
kseries = np.reshape(np.linspace(-4.,4.,nknum),(1,nknum))

gaverage = 0.6
xee,xei,xie,xii=1.0,1.0,1.0,1.0 

## for all casesq
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]

''' ## Three \bar{J} cases '''
JI,JE,a,b=0.575+1.0,1.325+1.0,0,0#0.575+1.0,1.325+1.75,0,0#
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
''' for reference, eigenvectors of unperturbed matrix \bar{J}'''
xAm,yAm=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))

mAm,nAm = xAm.copy()*np.sqrt(N),yAm.copy()*np.sqrt(N)*eigvAm[0]
muxgeneral,muygeneral = mAm.copy(),nAm.copy()

''' ----------- Gaussian ---------  ''' 
Xsym=np.random.randn(NE*2,NE*2)*gaverage/np.sqrt(2*NE) # /np.sqrt(2) no later
Xsym[:NE,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
Xsym[:NE,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
Xsym[NE:,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
Xsym[NE:,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
XsymT=Xsym.copy().T
X0=Xsym.copy()
J0=Am.copy()+X0.copy()
''' # calculate the original \bar{m} and \bar{n} '''
eigvJ0,eigrvecJ0=la.eig(J0)
inveigrvecJ0=la.inv(eigrvecJ0)
meig0 = np.squeeze(eigrvecJ0[:,:].copy())
neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
neig0=neig0.copy().T
for i in range(nrank):
    neig0[:,i]*=eigvJ0[i]
for j in range(nrank,N):
    neig0[:,j]*=eigvJ0[j]

leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
''' The first '''
reig0=meig0.copy()
normval=np.sum(reig0*neig0.copy(),axis=0)
normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
leig0=neig0.copy()/normval.copy()
## for reference
tildex,tildey=np.reshape(reig0[:,0].copy(),(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
## make sure the E in y is positive, for negative change signs of x and y
if (np.mean(tildey[:NE,0])*yAm[0,0])<0:
    tildex*=(-1)
    tildey*=(-1)
x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey
m0,n0 = np.reshape(x0.copy()*np.sqrt(NE*2),(2*NE,1)), np.reshape(y0.copy()*np.sqrt(2*NE)*eigvJ0[0],(NE*2,1))


''' theorem '''
sigm,sign = np.zeros(2),np.zeros(2)
sigm[:]=gaverage**2/eigvAm[0]**2
sign[:]=2*(JE**2+JI**2)*gaverage**2/(eigvAm[0])**2
dxactrange = np.linspace(-3*sigm[0],3*sigm[0],NE*2)
gaussdis = np.exp(-(dxactrange)**2/2/sigm[0])
gaussdis = np.reshape(gaussdis/np.sum(gaussdis),(NE*2,1))
gaussdis = np.repeat(gaussdis,nknum,axis=1)
mxactrange = np.reshape(dxactrange,(2*NE,1)) + mAm
mxactrange = mxactrange@kseries
derivtheo = gradtanh(mxactrange)
muderivtheo = np.sum(derivtheo*gaussdis,axis=0)
sigmathrough_theo = np.reshape(muderivtheo,(1,nknum))*kseries
## --------------------------------------------
muderivnum = np.mean(gradtanh(m0@kseries),axis=0)
sigmathrough_num = np.reshape(muderivnum,(1,nknum))*kseries

## mean phi
phixact = np.tanh(mxactrange)
muphiacttheo =  np.sum(phixact*gaussdis,axis=0)
anpthrough_theo = 0.5*(nAm[0,0]*muphiacttheo[:]+nAm[NE+1,0]*muphiacttheo[:])
muphiactnum = np.mean(np.tanh(m0@kseries),axis=0)
anpthrough_num = 0.5*(np.mean(n0[:NE,0])*muphiactnum[:]+np.mean(n0[NE:,0])*muphiactnum[:])


<ipython-input-26-89f12ab13b61>:36: RuntimeWarning: divide by zero encountered in true_divide
  neigvecAm[:,i]/=normalizecoe
<ipython-input-26-89f12ab13b61>:36: RuntimeWarning: invalid value encountered in true_divide
  neigvecAm[:,i]/=normalizecoe
<ipython-input-26-89f12ab13b61>:83: ComplexWarning: Casting complex values to real discards the imaginary part
  sigm[:]=gaverage**2/eigvAm[0]**2
<ipython-input-26-89f12ab13b61>:84: ComplexWarning: Casting complex values to real discards the imaginary part
  sign[:]=2*(JE**2+JI**2)*gaverage**2/(eigvAm[0])**2


In [32]:
xticks = np.linspace(-2.0,2.0,2)
xlims = [-2.0,2.0]
yticks = np.linspace(-2.0,2.0,2)
ylims = [-2.0,2.0]

fig,ax=plt.subplots(figsize=(4,4))   
ax.plot(kseries[0,:],np.real(muphiacttheo[:]),linestyle='-',color='gray',linewidth=1.5)   
# ax.plot(kseries[0,:],np.real(muphiactnum[:]),linestyle='--',color='blue',linewidth=1.5)   
ax.plot(kseries[0,:],np.real(muderivtheo[:]),linestyle='-',color='black',linewidth=1.5)  
# ax.plot(kseries[0,:],np.real(muderivnum[:]),linestyle='--',color='blue',linewidth=1.5)  
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')

fig,ax=plt.subplots(figsize=(4,4))    
ax.plot(kseries[0,:],np.real(kseries[0,:]),linestyle='--',color='grey')  
ax.plot(kseries[0,:],np.real(sigmathrough_theo[0,:]),linestyle='-',color='black',linewidth=1.5) 
# ax.plot(kseries[0,:],np.real(sigmathrough_num[0,:]),linestyle='-',color='blue',linewidth=1.5) 
ax.plot(kseries[0,:],np.real(anpthrough_theo[:]),linestyle='-',color='grAy',linewidth=1.5) 
# ax.plot(kseries[0,:],np.real(anpthrough_num[:]),linestyle='--',color='blue',linewidth=1.5) 
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')